In [2]:
import math
import random
import itertools

import numpy as np
import numba
import numpy.random
from collections import deque

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

In [3]:
class GridWorld:
    def __init__(self, taille, position_start, good_end_position, bad_end_position):
        self.current_state = position_start  # État actuel (ligne, colonne)
        self.states = [[x, y] for x in range(taille[0]) for y in range(taille[1])]
        self.end_good_state = good_end_position  # État final (ligne, colonne)
        self.end_bad_state = bad_end_position
        self.grid_size = taille  # Taille de la grille (lignes, colonnes)
        self.stateSpace = {}
        self.matchStates()
        self.currentIntState = self.getStateInt(self.current_state)
        self.num_actions = 4  # Nombre total d'actions possibles (haut, bas, gauche, droite)
        self.reward = 0  # Récompense actuelle
        self.done = False  # Indique si la partie est terminée
        self.generate_grid()
        self.actions = [0, 1, 2, 3]
        self.rewards = [0, 1, 3]
        self.actionSpace = {0: -self.grid_size[0], 1: self.grid_size[0],
                            2: -1, 3: 1}
        self.P = {}

    
    def reset(self, position_start):
        self.current_state = position_start
        self.currentIntState = self.getStateInt(position_start)
        
    def matchStates(self):
        i=0
        for s in self.states:
            self.stateSpace[str(s)] = i
            i = i+1
    
    def getStateInt(self, st):
        return self.stateSpace[str(st)]
    
    def getStateCouple(self, st):
        n_state = {i for i in self.stateSpace if self.stateSpace[i]==st}
        return list(n_state)
        
    def initP(self):
        for state in self.states:
            st = self.getStateInt(state)
            for action in self.actions:
                self.current_state = state
                state_, reward_, done_ = self.step(action)
                stt = self.getStateInt(state_)
                self.P[(stt, reward_, st, action)] = 1

    
    def step(self, action):
        if action == 0:
            if self.current_state[0] == 0 :
                self.current_state[0] = self.grid_size[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour traverser le mur
                self.generate_grid()
                self.endgame()
            else:
                self.current_state[0] = self.current_state[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()

        elif action == 1:
            if self.current_state[0] == self.grid_size[0] - 1:
                self.current_state[0] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()
            else :
                self.current_state[0] = self.current_state[0] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()

        elif action == 2:
            if self.current_state[1] == 0:
                self.current_state[1] = self.grid_size[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()

        elif action == 3:
            if self.current_state[1] == self.grid_size[1] - 1:
                self.current_state[1] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                #print(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()
                # Si l'on atteint l'état final, la partie est terminée
        return self.current_state, self.reward, self.done

    def endgame(self):
        if self.current_state == self.end_good_state:
            self.reward = 10  # Récompense de 1 pour atteindre l'état final
            self.done = True
        elif self.current_state == self.end_bad_state:
            self.reward = -10
            self.done = True

    def generate_grid(self):
        grid=[]
        for i in range(self.grid_size[0]):
            grid.append([])
            for j in range(self.grid_size[1]):
                grid[i].append("_")
        grid[self.current_state[0]][self.current_state[1]] = "X"
        for i in grid:
            print(i)
        print("\n")
    
    def printQ(self, Q):
        for key in Q:
            print(str(key) + " : \t")
            for val in Q[key]:
                print(str(Q[key][val]) + "\t")

In [4]:
class DQN_Agent:
    def __init__(self, environment):
        self.env = environment
        self.actions = environment.actions
        self.state_size = len(environment.states)
        self.action_size = len(environment.actions)
        self.memory = deque(maxlen=2000)
        state = environment.current_state
        self.batch_size = 32
        q_network = self.build_compile_model()
        target_network = self.build_compile_model()
        
        self.alpha = 0.1
        self.epsilon = 0.05
        self.discount_factor = 0.9
        
        self.model = self.model_building()
        self.height = environment.grid_size[0]
        self.width = environment.grid_size[1]

        
    def build_compile_model(self):
        model = Sequential()
        model.add(Embedding(self.state_size, 10, input_length=1))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.01))
        return model
  
    def memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.actions)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

        
    def retrain(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.q_network.predict(np.reshape(state, [1, 1]))
            if done:
                target[0][action] = reward
            else:
                t = self.target_network.predict(np.reshape(next_state, [1, 1]))
                target[0][action] = reward + self.discount_factor * np.amax(t)
            self.q_network.fit(np.reshape(state, [1, 1]), target, epochs=1, verbose=0)

In [5]:
def build_compile_model(grid):
    model = Sequential()
    model.add(Embedding(len(grid.states), 10, input_length=1))
    model.add(Reshape((10,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(len(grid.actions), activation='linear'))

    model.compile(loss='mse', optimizer=Adam(learning_rate=0.01))
    return model

In [6]:
def q_learning(grid, max_iter_count = 100, discount_factor=0.9):
    epsilon = 0.05
    alpha = 0.1
    max_steps_per_episode = 1000
    int_state = np.reshape(grid.currentIntState, [1, 1])
    state = grid.current_state
    batch_size = 32
    memory = deque(maxlen=2000)
    q_network = build_compile_model(grid)
    target_network = build_compile_model(grid)
    grid_height = grid.grid_size[0]
    grid_width = grid.grid_size[1]
    
    reward_per_episode = []
            
    
    for episode in range(max_iter_count):
        grid.__init__([grid_height, grid_width], [grid_height-1, np.random.randint(0,grid_width)], grid.end_good_state, grid.end_bad_state)
        
        current_state = grid.current_state
        current_Int_State = np.reshape(grid.currentIntState, [1, 1])
        
        print("Episode : ", episode)
        reward = 0
        step = 0
        game_done = False
        cumulative_reward = 0
        
        while (step<500) and (game_done!=True):  
            print(step)
            actions = grid.actions
            
            if np.random.uniform(0,1) <= epsilon:
                a = np.random.choice(actions)
            else:
                q_values = q_network.predict(current_Int_State)
                a= np.argmax(q_values[0])

             
            old_reward   = grid.reward
            grid.step(a)
            reward   = grid.reward
            diff_reward  = reward - old_reward
            n_state = grid.current_state
            n_Int_state = grid.currentIntState
            n_actions = grid.actions
            game_done = grid.done
            
            memory.append((int_state, a, reward, n_Int_state, game_done))
            
            int_state = n_Int_state
            state = n_state
            
            if game_done:
                print("Game done")
                target_network.set_weights(q_network.get_weights())
                break
            
            if (len(memory) > batch_size):
                minibatch = random.sample(memory, batch_size)
                for s, ac, reward, ns, terminated in minibatch:
                    target = q_network.predict(np.reshape(s, [1, 1]))
                    if terminated:
                        target[0][a] = reward
                    else:
                        t = target_network.predict(np.reshape(ns, [1, 1]))
                        target[0][a] = reward + discount_factor * np.amax(t)
                    q_network.fit(np.reshape(s, [1, 1]), target, epochs=1, verbose=0)
            
            cumulative_reward += reward
            step += 1
            
        reward_per_episode.append(cumulative_reward)
        print(reward_per_episode)
    return reward_per_episode
            

In [ ]:
if __name__ == '__main__':
    # Il faut choisir un nombre entre 0 et 3
    # Si l'action est 0, déplacement vers le haut
    # Si l'action est 1, déplacement vers le bas
    # Si l'action est 2, déplacement vers la gauche
    # Si l'action est 3, déplacement vers la droite
    world = GridWorld([4,4],[0,0],[3,3],[2,3])
    q_learning(world, max_iter_count = 100, discount_factor=0.9)

['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']


Episode :  0
0
1/1 [==============================] - 0s 464ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']


1
1/1 [==============================] - 0s 30ms/step
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


2
1/1 [==============================] - 0s 40ms/step
['_', '_', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


3
1/1 [==============================] - 0s 48ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']


4
1/1 [==============================] - 0s 47ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']


5
1/1 [==============================] - 0s 37ms/step
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_

1/1 [==============================] - 0s 44ms/step
34
1/1 [==============================] - 0s 47ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', 'X', '_', '_']


1/1 [==============================] - 0s 42ms/step
35
1/1 [==============================] - 0s 31ms/step
['_', 'X', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 37ms/step
36
1/1 [==============================] - 0s 65ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', 'X', '_', '_']


1/1 [==============================] - 0s 35ms/step
37
1/1 [==============================] - 0s 43ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['X', '_', '_', '_']


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 45ms/step
38
1/1 [==============================] - 0s 29ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', 'X']


Game done
[-38]
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['X', '_', '_', '_']


Episode :  1
0
1/1 [==============================] - 0s 47ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['X', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 36ms/step
1
1/1 [==============================] - 0s 44ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', 'X']
['_', '_', '_', '_']


Game done
[-38, -1]
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']


Episode :  2
0
1/1 [==============================] - 0s 55ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', 'X', '_', '_']


1/1 [==============================] - 0s 34ms/step
1
1/1 [==============================] - 0s 37ms/

1/1 [==============================] - 0s 46ms/step
2
1/1 [==============================] - 0s 50ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', 'X']


Game done
[-38, -1, -2]
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', 'X', '_', '_']


Episode :  3
0
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', 'X', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 63ms/step
1
1/1 [==============================] - 0s 45ms/step
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 33ms/step


...

1/1 [==============================] - 0s 50ms/step
34
1/1 [==============================] - 0s 37ms/step
['_', 'X', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 30ms/step
35
1/1 [==============================] - 0s 37ms/step
['_', '_', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 61ms/step
36
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 44ms/step
37
1/1 [==============================] - 0s 46ms/step
['_', '_', '_', '_']
['_', '_', '_', 'X']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 46ms/step
38
1/1 [==============================] - 0s 45ms/step
['_', '_', '_', '_']
['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']




1/1 [==============================] - 0s 31ms/step
39
1/1 [==============================] - 0s 37ms/step
['_', '_', '_', '_']
['_', 'X', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 44ms/step
40
1/1 [==============================] - 0s 47ms/step
['_', '_', '_', '_']
['_', '_', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 31ms/step
41
1/1 [==============================] - 0s 47ms/step
['_', '_', '_', '_']
['_', '_', '_', 'X']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 45ms/step
42
1/1 [==============================] - 0s 38ms/step
['_', '_', '_', '_']
['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


1/1 [==============================] - 0s 33ms/step
43
1/1 [==============================] - 0s 31ms/step
['_', '_', '_', '_']
['_', 'X', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']




1/1 [==============================] - 0s 34ms/step
44
